<a href="https://colab.research.google.com/github/DanielCob/accesible-gaussian-splatting/blob/main/accesible_gaussian_splatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gaussian Splatting - Accessible 3D Reconstruction

This notebook adapts the [original](https://github.com/graphdeco-inria/gaussian-splatting) reference implementation of "3D Gaussian Splatting for Real-Time Radiance Field Rendering" to create a simplified, educational and fully automated pipeline with a graphical interface for non-expert users.  
Developed by Daniel Cob Beirute, Instituto Tecnológico de Costa Rica (2025).

# Dependencies installation

In [ ]:
# Clone and install Guassian Splatting repository
%cd /content
!git clone --recursive https://github.com/DanielCob/accesible-gaussian-splatting.git
!pip install -q plyfile

%cd /content/accesible-gaussian-splatting
!pip install -q /content/accesible-gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/accesible-gaussian-splatting/submodules/simple-knn

In [ ]:
# Install dependencies
!sudo apt-get install -y colmap
!sudo apt-get install -y imagemagick
!sudo apt-get install -y xvfb # OpenGL workaround for Google Colab

# Data upload point
(upload a short video orbiting a object or scene)

In [ ]:
from google.colab import files
import os

uploaded = files.upload()
# The user uploads a single video file
uploaded_filename = list(uploaded.keys())[0]

# Create the input directory if it doesn't exist
input_dir = '/content/accesible-gaussian-splatting/data/video'
os.makedirs(input_dir, exist_ok=True)

# Define the new path for the video
video_path = os.path.join(input_dir, uploaded_filename)

# Move the uploaded video to the input directory
os.rename(uploaded_filename, video_path)

print("Video uploaded and moved to:", video_path)

# Preproccesing

In [ ]:
# Create a directory to store the extracted frames
!mkdir /content/accesible-gaussian-splatting/data/input

# Use ffmpeg to extract frames from the uploaded video
!ffmpeg -i "{video_path}" -qscale:v 1 -qmin 1 -vf fps=2 /content/accesible-gaussian-splatting/data/input/%04d.jpg

print("Frames extracted to /content/gaussian-splatting/data/input/")

In [ ]:
%cd /content/accesible-gaussian-splatting
!python convert.py -s /content/accesible-gaussian-splatting/data/ --resize

# Training

In [ ]:
%cd /content/accesible-gaussian-splatting
!python train.py -s /content/accesible-gaussian-splatting/data/

# Training with Evaluation (Train/Test split)

In [ ]:
%cd /content/accesible-gaussian-splatting
!python train.py -s /content/accesible-gaussian-splatting/data/ --eval
!python render.py -m /content/accesible-gaussian-splatting/data/
!python metrics.py -m /content/accesible-gaussian-splatting/data/

# Exporting output
(.zip can later be visualized using other visualization tools like the one provided by [Camenduru](https://colab.research.google.com/github/camenduru/gaussian-splatting-colab/blob/main/gaussian_splatting_viewer_colab.ipynb) )

In [ ]:
import shutil
import os
from google.colab import files

output_dir = '/content/accesible-gaussian-splatting/output'
compressed_output_path = '/content/accesible-gaussian-splatting/output.zip'

# Compress the output folder
shutil.make_archive(compressed_output_path.replace('.zip', ''), 'zip', output_dir)

# Provide a download link
files.download(compressed_output_path)

print(f"Output folder compressed to {compressed_output_path} and ready for download.")